In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error

import pickle

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/10/28 15:46:55 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/28 15:46:55 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/Users/kennethleo/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1761635392108, experiment_id='1', last_update_time=1761635392108, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)] #filtering

    categorical = ['PULocationID', 'DOLocationID'] # we can extract the day of week, hour of day, month, etc from the datetime columns too
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe('./data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2024-02.parquet')

In [5]:
len(df_train), len(df_val)

(54373, 51497)

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical =  ['PU_DO'] #['PULocationID', 'DOLocationID'] # we can extract the day of week, hour of day, month, etc from the datetime columns too
numerical = ['trip_distance']

dv = DictVectorizer() # turns dictionary to a vector
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)


In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values   

In [9]:
linear = LinearRegression()
linear.fit(X_train, y_train)

y_pred = linear.predict(X_val)

RMSE = root_mean_squared_error(y_val, y_pred)
RMSE

5.994257895960117

In [10]:
lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)

RMSE = root_mean_squared_error(y_val, y_pred)
RMSE

9.110340896856446

In [11]:
ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_val)

RMSE = root_mean_squared_error(y_val, y_pred)
RMSE

5.9788341587284455

In [12]:
#saving the model
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, linear), f_out)

with open('models/lasso.bin', 'wb') as f_out:
    pickle.dump((dv, lasso), f_out)

with open('models/ridge.bin', 'wb') as f_out:
    pickle.dump((dv, ridge), f_out)

Using ML Flow

In [26]:
with mlflow.start_run():

    mlflow.set_tag("developer", "kenneth")
    mlflow.log_param("train-data-path" ,"./data/green_tripdata_2024-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2024-02.parquet")

    alpha = 0.1

    mlflow.log_param("alpha", alpha)

    lasso = Lasso(alpha)
    lasso.fit(X_train, y_train)

    y_pred = lasso.predict(X_val)

    RMSE = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric("RMSE", RMSE)

    #saving model as an artifact
    mlflow.log_artifact(local_path="models/lasso.bin", artifact_path="models_pickle")

Using XGBoost with MLFlow

In [15]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [16]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

[0]	validation-rmse:6.25059                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:53:10] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:5.55563                           
[2]	validation-rmse:5.40343                           
[3]	validation-rmse:5.34169                           
[4]	validation-rmse:5.32371                           
[5]	validation-rmse:5.31657                           
[6]	validation-rmse:5.31469                           
[7]	validation-rmse:5.31063                           
[8]	validation-rmse:5.30493                           
[9]	validation-rmse:5.30372                           
[10]	validation-rmse:5.30073                          
[11]	validation-rmse:5.28990                          
[12]	validation-rmse:5.28942                          
[13]	validation-rmse:5.28590                          
[14]	validation-rmse:5.28573                          
[15]	validation-rmse:5.28501                          
[16]	validation-rmse:5.27331                          
[17]	validation-rmse:5.27293                          
[18]	validation-rmse:5.27120                          
[19]	valid

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:53:15] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



...

[1]	validation-rmse:6.95443                                                     
[2]	validation-rmse:6.37285                                                     
[3]	validation-rmse:5.99631                                                     
[4]	validation-rmse:5.75660                                                     
[5]	validation-rmse:5.60291                                                     
[6]	validation-rmse:5.50718                                                     
[7]	validation-rmse:5.44663                                                     
[8]	validation-rmse:5.40670                                                     
[9]	validation-rmse:5.38196                                                     
[10]	validation-rmse:5.36365                                                    
[11]	validation-rmse:5.35170                                                    
[12]	validation-rmse:5.34133                                                    
[13]	validation-rmse:5.33459

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:04:28] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.26844                                                     
[1]	validation-rmse:6.26299                                                     
[2]	validation-rmse:5.74164                                                     
[3]	validation-rmse:5.48355                                                     
[4]	validation-rmse:5.35401                                                     
[5]	validation-rmse:5.28514                                                     
[6]	validation-rmse:5.25155                                                     
[7]	validation-rmse:5.23139                                                     
[8]	validation-rmse:5.21809                                                     
[9]	validation-rmse:5.20845                                                     
[10]	validation-rmse:5.20339                                                    
[11]	validation-rmse:5.20032                                                    
[12]	validation-rmse:5.19684

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:04:32] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.40906                                                     
[1]	validation-rmse:7.82733                                                     
[2]	validation-rmse:7.35527                                                     
[3]	validation-rmse:6.94624                                                     
[4]	validation-rmse:6.62737                                                     
[5]	validation-rmse:6.36796                                                     
[6]	validation-rmse:6.14591                                                     
[7]	validation-rmse:5.97831                                                     
[8]	validation-rmse:5.84747                                                     
[9]	validation-rmse:5.74383                                                     
[10]	validation-rmse:5.64635                                                    
[11]	validation-rmse:5.58191                                                    
[12]	validation-rmse:5.52849

In [24]:
params = {
    'learning_rate':  0.13079192861893357,
    'max_depth': 14,
    'min_child_weight': 1.9983067958652456,
    'objective': 'reg:linear',
    'reg_alpha': 0.00972710129657474,
    'reg_lambda': 0.006371719563088103,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2025/10/28 20:06:53 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '30e1d9be39644a4f99ce1256b7765380', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:8.37173
[1]	validation-rmse:7.76520
[2]	validation-rmse:7.27372


/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [20:06:54] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[3]	validation-rmse:6.87620
[4]	validation-rmse:6.55911
[5]	validation-rmse:6.30213
[6]	validation-rmse:6.10141
[7]	validation-rmse:5.94399
[8]	validation-rmse:5.81766
[9]	validation-rmse:5.71931
[10]	validation-rmse:5.64404
[11]	validation-rmse:5.58350
[12]	validation-rmse:5.53219
[13]	validation-rmse:5.49299
[14]	validation-rmse:5.46201
[15]	validation-rmse:5.43474
[16]	validation-rmse:5.41329
[17]	validation-rmse:5.39644
[18]	validation-rmse:5.38076
[19]	validation-rmse:5.36863
[20]	validation-rmse:5.35995
[21]	validation-rmse:5.35385
[22]	validation-rmse:5.34820
[23]	validation-rmse:5.34160
[24]	validation-rmse:5.33449
[25]	validation-rmse:5.33095
[26]	validation-rmse:5.32816
[27]	validation-rmse:5.32485
[28]	validation-rmse:5.32246
[29]	validation-rmse:5.32116
[30]	validation-rmse:5.32047
[31]	validation-rmse:5.31772
[32]	validation-rmse:5.31663
[33]	validation-rmse:5.31434
[34]	validation-rmse:5.31208
[35]	validation-rmse:5.31168
[36]	validation-rmse:5.31090
[37]	validation-rmse:

2025/10/28 20:07:11 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/10/28 20:07:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 20:07:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [20:07:11] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/10/28 20:07:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [28]:
mlflow.xgboost.autolog(disable=True)

In [29]:
with mlflow.start_run():
    best_params = {
        'learning_rate':  0.13079192861893357,
        'max_depth': 14,
        'min_child_weight': 1.9983067958652456,
        'objective': 'reg:linear',
        'reg_alpha': 0.00972710129657474,
        'reg_lambda': 0.006371719563088103,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")



[0]	validation-rmse:8.37173


/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:47:05] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:7.76520
[2]	validation-rmse:7.27372
[3]	validation-rmse:6.87620
[4]	validation-rmse:6.55911
[5]	validation-rmse:6.30213
[6]	validation-rmse:6.10141
[7]	validation-rmse:5.94399
[8]	validation-rmse:5.81766
[9]	validation-rmse:5.71931
[10]	validation-rmse:5.64404
[11]	validation-rmse:5.58350
[12]	validation-rmse:5.53219
[13]	validation-rmse:5.49299
[14]	validation-rmse:5.46201
[15]	validation-rmse:5.43474
[16]	validation-rmse:5.41329
[17]	validation-rmse:5.39644
[18]	validation-rmse:5.38076
[19]	validation-rmse:5.36863
[20]	validation-rmse:5.35995
[21]	validation-rmse:5.35385
[22]	validation-rmse:5.34820
[23]	validation-rmse:5.34160
[24]	validation-rmse:5.33449
[25]	validation-rmse:5.33095
[26]	validation-rmse:5.32816
[27]	validation-rmse:5.32485
[28]	validation-rmse:5.32246
[29]	validation-rmse:5.32116
[30]	validation-rmse:5.32047
[31]	validation-rmse:5.31772
[32]	validation-rmse:5.31663
[33]	validation-rmse:5.31434
[34]	validation-rmse:5.31208
[35]	validation-rmse:5.

2025/10/28 23:47:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:47:28] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/10/28 23:47:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [37]:
#try running the saved model
logged_model = 'runs:/5ef86dad014342e59f774415cb739edc/models_mlflow'

#load model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [32]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: /Users/kennethleo/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/mlruns/1/models/m-b8895dc6c0414bd2b91ad675d593cd7e/artifacts
  flavor: mlflow.xgboost
  run_id: 5ef86dad014342e59f774415cb739edc

In [33]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [38]:
with mlflow.start_run(run_id="5ef86dad014342e59f774415cb739edc"):
    mlflow.xgboost.log_model(
        xgb_model=booster,
        artifact_path="model"
    )

2025/10/29 11:43:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:43:20] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/10/29 11:43:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [35]:
y_pred = xgboost_model.predict(valid)

In [36]:
y_pred[:10]

array([19.223265 , 27.983643 ,  7.878154 ,  9.024201 ,  4.3738728,
        4.499487 ,  9.753556 , 17.45656  , 22.446112 ,  6.813442 ],
      dtype=float32)